In [1]:
# Imports dan paths

import pandas as pd, numpy as np, optuna, mlflow, xgboost as xgb
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
# Load data
DAILY_PARQUET = Path("../data/processed/daily_features.parquet")
df_daily = pd.read_parquet(DAILY_PARQUET)
df_daily.head()

,station_id,date_local,temp_13LT_C,rh_avg_pc,wind_avg_kmh,qff_avg_hPa,rain_mm,evap_mm,ffmc
0,96595,2015-01-01 00:00:00+00:00,32.2,71.39375,0.92600,1009.58750,3.0,4.0,73.688695
1,96595,2015-01-02 00:00:00+00:00,30.0,79.08750,0.92600,1010.58125,0.0,3.0,83.174782
2,96595,2015-01-03 00:00:00+00:00,31.1,79.30000,0.00000,1010.85625,0.0,3.0,84.831423
3,96595,2015-01-04 00:00:00+00:00,30.9,76.87500,1.15750,1009.85000,20.5,6.0,60.330249
4,96595,2015-01-05 00:00:00+00:00,29.2,85.48750,0.57875,1010.38750,10.8,6.0,58.220215


In [3]:
df_daily["date_local"] = pd.to_datetime(df_daily["date_local"])

In [4]:
### Feature engineering – lags, rolling means, DOY encodings
LAGS  = [1, 3]
ROLLS = [3, 7]

df = df_daily.sort_values(["station_id", "date_local"]).set_index("date_local")

for col in ["temp_13LT_C", "rh_avg_pc", "wind_avg_kmh",
            "qff_avg_hPa", "rain_mm", "evap_mm"]:
    for k in LAGS:
        df[f"{col}_lag{k}"] = df.groupby("station_id")[col].shift(k)
    for w in ROLLS:
        df[f"{col}_roll{w}"] = (
            df.groupby("station_id")[col]
              .rolling(w, min_periods=1).mean()
              .droplevel(0)
        )

df["doy_sin"] = np.sin(2*np.pi*df.index.dayofyear / 365.25)
df["doy_cos"] = np.cos(2*np.pi*df.index.dayofyear / 365.25)
df["month"]   = df.index.month

# drop first-lag rows that now contain NaN
df = df.dropna(subset=[c for c in df.columns if "lag" in c]).reset_index()
df.head()


,date_local,station_id,temp_13LT_C,rh_avg_pc,wind_avg_kmh,qff_avg_hPa,rain_mm,evap_mm,ffmc,temp_13LT_C_lag1,...,rain_mm_lag3,rain_mm_roll3,rain_mm_roll7,evap_mm_lag1,evap_mm_lag3,evap_mm_roll3,evap_mm_roll7,doy_sin,doy_cos,month
0,2015-01-04 00:00:00+00:00,96595,30.9,76.87500,1.15750,1009.85000,20.5,6.0,60.330249,31.1,...,3.0,6.833333,5.875000,3.0,4.0,4.000000,4.000000,0.068755,0.997634,1
1,2015-01-05 00:00:00+00:00,96595,29.2,85.48750,0.57875,1010.38750,10.8,6.0,58.220215,30.9,...,0.0,10.433333,6.860000,6.0,3.0,5.000000,4.400000,0.085906,0.996303,1
2,2015-01-06 00:00:00+00:00,96595,30.0,82.90625,1.50475,1009.46250,32.5,6.0,60.075054,29.2,...,0.0,21.266667,11.133333,6.0,3.0,6.000000,4.666667,0.103031,0.994678,1
3,2015-01-07 00:00:00+00:00,96595,27.2,88.31875,2.08350,1010.45625,19.7,4.0,56.740650,30.0,...,20.5,21.000000,12.357143,6.0,6.0,5.333333,4.571429,0.120126,0.992759,1
4,2015-01-08 00:00:00+00:00,96595,28.6,84.60625,1.15750,1010.61250,6.3,4.0,62.820614,27.2,...,10.8,19.500000,12.828571,4.0,6.0,4.666667,4.571429,0.137185,0.990545,1


In [8]:
###  NEW – station-stratified random split

from sklearn.utils import shuffle

test_frac = 0.20
test_idx  = (
    df.groupby('station_id', group_keys=False)
      .apply(lambda g: g.sample(frac=test_frac, random_state=42))
      .index
)

train_idx = df.index.difference(test_idx)

train = df.loc[train_idx].copy()
test  = df.loc[test_idx].copy()

PRED_COLS = [c for c in df.columns if c not in ('station_id', 'ffmc', 'date_local')]
X_train, y_train = train[PRED_COLS], train['ffmc']
X_test,  y_test  = test[PRED_COLS],  test['ffmc']
print(f"Train rows: {len(train):,}  |  Test rows: {len(test):,}")

print(y_train[:10])
print(y_test[:10])


Train rows: 13,095  |  Test rows: 3,273
1     58.220215
2     60.075054
3     56.740650
4     62.820614
5     81.446023
6     65.017678
7     60.586865
8     53.526273
9     60.842692
10    50.632887
Name: ffmc, dtype: float64
2210    75.815924
2338    84.784616
1656    83.212600
1652    79.109409
1451    59.156528
2638    67.484119
864     80.991939
238     86.612615
2576    56.095540
1366    83.784558
Name: ffmc, dtype: float64


/tmp/ipykernel_7263/816413710.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(frac=test_frac, random_state=42))


In [9]:
### Optuna + MLflow tuning – find best XGBoost hyper-params
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("ffmc_xgb")

def objective(trial):
    params = {
        "n_estimators"     : trial.suggest_int("n_estimators", 300, 1200),
        "max_depth"        : trial.suggest_int("max_depth", 3, 10),
        "learning_rate"    : trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample"        : trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree" : trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "random_state"     : 42,
        "n_jobs"           : -1,
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred))

    with mlflow.start_run():
        mlflow.log_params(params)
        mlflow.log_metric("rmse", rmse)
        mlflow.xgboost.log_model(model, "model")
    return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40, show_progress_bar=True)

best_rmse = study.best_value
print("Best RMSE:", best_rmse)


[I 2025-06-22 10:55:14,658] A new study created in memory with name: no-name-aaead55b-f3d8-44f5-a596-b06beadc5458


  0%|          | 0/40 [00:00<?, ?it/s]

2025/06/22 10:55:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/rzby/miniforge3/envs/atmo-ann/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [10:55:22] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1748293041487/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/06/22 10:55:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[I 2025-06-22 10:55:25,386] Trial 0 finished with value: 0.47994708796528074 and parameters: {'n_estimators': 1140, 'max_depth': 7, 'learning_rate': 0.01769006525284449, 'subsample': 0.7562825182548983, 'colsample_bytree': 0.753874811087426}. Best is trial 0 with value: 0.47994708796528074.


2025/06/22 10:55:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/rzby/miniforge3/envs/atmo-ann/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [10:55:43] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1748293041487/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/06/22 10:55:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

[I 2025-06-22 10:59:43,716] Trial 37 finished with value: 0.5566527080965179 and parameters: {'n_estimators': 851, 'max_depth': 10, 'learning_rate': 0.044722257266827145, 'subsample': 0.650040321230466, 'colsample_bytree': 0.89671750067093}. Best is trial 14 with value: 0.39086946679819357.


2025/06/22 10:59:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/rzby/miniforge3/envs/atmo-ann/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [10:59:48] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1748293041487/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/06/22 10:59:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[I 2025-06-22 10:59:50,592] Trial 38 finished with value: 0.4213552754993032 and parameters: {'n_estimators': 1144, 'max_depth': 6, 'learning_rate': 0.020146949466170894, 'subsample': 0.6804123938470519, 'colsample_bytree': 0.8210915642434258}. Best is trial 14 with value: 0.39086946679819357.


2025/06/22 10:59:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/rzby/miniforge3/envs/atmo-ann/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [10:59:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1748293041487/work/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/06/22 10:59:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[I 2025-06-22 10:59:54,239] Trial 39 finished with value: 0.39346980817920835 and parameters: {'n_estimators': 920, 'max_depth': 4, 'learning_rate': 0.056033475659544814, 'subsample': 0.9880709110323536, 'colsample_bytree': 0.9360306823710991}. Best is trial 14 with value: 0.39086946679819357.
Best RMSE: 0.39086946679819357


In [10]:
### Retrieve best model & evaluate RQ 1 metrics
best_run = mlflow.search_runs(order_by=["metrics.rmse"], max_results=1).iloc[0]
model_uri = f"runs:/{best_run.run_id}/model"
best_model = mlflow.xgboost.load_model(model_uri)

test["pred"] = best_model.predict(X_test)

overall = {
    "RMSE": np.sqrt(mean_squared_error(y_test, test["pred"])),
    "MAE" : mean_absolute_error(y_test, test["pred"]),
    "R2"  : r2_score(y_test, test["pred"]),
}
print("=== Overall test metrics ===")
for k, v in overall.items():
    print(f"{k:4s}: {v:6.3f}")

per_station = (
    test.groupby("station_id")
        .apply(lambda g: pd.Series({
            "RMSE": np.sqrt(mean_squared_error(g["ffmc"], g["pred"])),
            "MAE" : mean_absolute_error(g["ffmc"], g["pred"]),
            "R2"  : r2_score(g["ffmc"], g["pred"]),
        }))
        .sort_index()
)
per_station


=== Overall test metrics ===
RMSE:  0.248
MAE :  0.179
R2  :  0.999


/tmp/ipykernel_7263/4069279142.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,RMSE,MAE,R2
station_id,,,
96595,0.209964,0.163368,0.999590
96645,0.369366,0.271962,0.998253
96651,0.233425,0.175150,0.999200
96653,0.189476,0.141887,0.999471
96655,0.195548,0.143170,0.999341


In [11]:
### ⬛ ANN – build, train, predict  (CELL 5-A)

from tensorflow import keras
from sklearn.preprocessing import StandardScaler

# 1. scale predictors
scaler_ann = StandardScaler().fit(X_train)
Xtr_ann = scaler_ann.transform(X_train)
Xte_ann = scaler_ann.transform(X_test)

# 2. build model
ann = keras.Sequential([
    keras.layers.Dense(128, activation="relu", input_shape=(Xtr_ann.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(1)         # linear output
])
ann.compile(loss="mse", optimizer=keras.optimizers.Adam(1e-3), metrics=["mae"])

# 3. train with early stopping
callback = keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True)
history = ann.fit(Xtr_ann, y_train,
                  validation_split=0.2,
                  epochs=300,
                  batch_size=128,
                  verbose=0,
                  callbacks=[callback])

# 4. predict
test["pred_ann"] = ann.predict(Xte_ann, verbose=0).ravel()


2025-06-22 11:18:05.566252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/rzby/miniforge3/envs/atmo-ann/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
### ⬛ SVM (RBF) – scale, tune C & γ quickly, predict  (CELL 5-B)

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

pipe = make_pipeline(StandardScaler(), SVR(kernel="rbf"))

param_grid = {
    "svr__C":      [1, 10, 100],
    "svr__gamma":  ["scale", 0.01, 0.1],
    "svr__epsilon":[0.1, 0.2],
}
grid = GridSearchCV(pipe, param_grid, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1)
grid.fit(X_train, y_train)

print("Best SVM params:", grid.best_params_)
test["pred_svm"] = grid.best_estimator_.predict(X_test)


Best SVM params: {'svr__C': 100, 'svr__epsilon': 0.2, 'svr__gamma': 0.01}


In [13]:
### ▶▶ Collect overall & per-station metrics for all three models  (CELL 6 updated)

def metric_dict(y_true, y_hat):
    return {
        "RMSE": np.sqrt(mean_squared_error(y_true, y_hat)),
        "MAE" : mean_absolute_error(y_true, y_hat),
        "R2"  : r2_score(y_true, y_hat),
    }

overall = pd.DataFrame({
    "XGB" : metric_dict(y_test, test["pred"]),
    "ANN" : metric_dict(y_test, test["pred_ann"]),
    "SVM" : metric_dict(y_test, test["pred_svm"]),
}).T
print("=== Overall test-set metrics ===")
overall


=== Overall test-set metrics ===


,RMSE,MAE,R2
XGB,0.248412,0.178966,0.999206
ANN,0.699215,0.516312,0.993706
SVM,2.701973,1.895916,0.906013


In [16]:
test[["ffmc", "pred", "pred_ann", "pred_svm"]].to_csv(Path("../reports/output_table.csv"), index=False)

In [ ]:
### Per-station RMSE for each model  (CELL 6-bis)

per_station_all = (
    test.groupby("station_id")
        .apply(lambda g: pd.Series({
            "XGB_RMSE": np.sqrt(mean_squared_error(g["ffmc"], g["pred"])),
            "ANN_RMSE": np.sqrt(mean_squared_error(g["ffmc"], g["pred_ann"])),
            "SVM_RMSE": np.sqrt(mean_squared_error(g["ffmc"], g["pred_svm"])),
        }))
)
per_station_all


In [ ]:
### Visual 3 – overall RMSE comparison  (CELL 7-bis)

overall.RMSE.plot.bar(rot=0, figsize=(4,3))
plt.ylabel("RMSE"); plt.title("Model comparison – overall RMSE")
plt.tight_layout(); plt.savefig(Path("../reports/figures/rmse_comparison.png"))
plt.show()

In [ ]:
### Visual 4 – per-station RMSE heatmap  (CELL 8-bis)
import seaborn as sns

plt.figure(figsize=(6,5))
sns.heatmap(per_station_all.filter(like="_RMSE").T,
            annot=True, fmt=".2f", cmap="YlOrRd")
plt.title("RMSE by station & model"); plt.xlabel("WMO station"); plt.ylabel("")
plt.tight_layout()
plt.savefig(Path("../reports/figures/rmse_station_comparison.png"))
plt.show()


In [ ]:
### Visual 1 – prediction vs observation scatter (CELL 7)
plt.figure(figsize=(5,5))
plt.scatter(test["ffmc"], test["pred"], s=10, alpha=0.5)
plt.plot([60,100], [60,100], "k--")
plt.xlabel("Observed FFMC"); plt.ylabel("Predicted FFMC")
plt.title("Overall test set – XGBoost")
plt.grid(True); plt.tight_layout()
plt.savefig(Path("../reports/figures/scatter_xgboost.png"))
plt.show()


In [ ]:
### RQ 2 – SHAP feature importance  (CELL 9)
import shap
sample = df.sample(6000, random_state=0)   # speed; adjust if GPU available
X_sample = sample[PRED_COLS]

explainer = shap.Explainer(best_model)
shap_values = explainer(X_sample, check_additivity=False)

shap.summary_plot(shap_values, X_sample, show=False)
plt.title("SHAP summary – key FFMC drivers")
plt.tight_layout()
plt.savefig(Path("../reports/figures/feature_xgboost.png"))
plt.show()
